<center> <h1> <b>Dev.Ensina - Modelos de Precificação de Opções</b> </h1> </center>

*C:* Valor da Opção (Call) <br>
*P:* Valor da Opção (Put)

*S:* Preço Atual do Ativo <br>
*X:* Preço de Exercício (Strike) <br>
*r:* Taxa de Livre de Risco <br>
*T:* Tempo até o vencimento <br>
*σ:* Volatilidade - Desvio-padrão dos Log-Retornos

*Obs:* Usaremos T em dias, logo r e σ serão valores diários.

In [20]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import minimize_scalar
import numpy as np
import requests
import plotly.graph_objects as go
from ipywidgets import interact

In [21]:
ticker = 'VALE3.SA'  # input('Digite o ticker do ativo de interesse: ')
T = 50  # int(input(f'Em quanto tempo deseja exercer sua opção (em dias)? '))

start = '2000-01-01'
end = datetime.now().strftime('%Y-%m-%d')

try:
    df = pd.DataFrame(yf.Ticker(ticker).history(start=start, end=end)["Adjusted Close"])
    df['Returns'] = [None] + [np.log((df['Adjusted Close'].iloc[i+1] / df['Adjusted Close'].iloc[i])) for i in range(len(df)-1)]
    S = df['Adjusted Close'].iloc[-1]
except:
    df = pd.DataFrame(yf.Ticker(ticker).history(start=start, end=end)["Close"])
    df['Returns'] = [None] + [np.log((df['Close'].iloc[i+1] / df['Close'].iloc[i])) for i in range(len(df)-1)]
    S = df['Close'].iloc[-1]
df.dropna(inplace=True)

X = S # float(input(f'Sendo {S} o valor atual do ativo, qual o preço de exercício da opção (strike price)? '))

sigma = df['Returns'].std()
mi = 0 # df['Returns'].mean()

r = 0 # float(requests.get('https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados/ultimos/1?formato=json').json()[0]['valor']) / 100

## **1) Modelo de Bachelier**

In [22]:
D = (S-X)/(sigma*np.sqrt(T))
C_bachelier = (S-X)*norm.cdf(D) + sigma*np.sqrt(T)*norm.pdf(D)

## **2) Modelo de Black-Scholes**

In [23]:
d1 = (np.log(S/X) + (r + (sigma**2)/2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)  # = np.log(S/X) + (r - (sigma**2)/2)*T / (sigma*np.sqrt(T))

C_BS = S*norm.cdf(d1) - X*np.exp(-r*T)*norm.cdf(d2)

## **3) Modelo de Freitas**

In [24]:
mi_ST = np.log(S) + T*mi
sigma_ST = np.sqrt(T)*sigma

def ret_lin(s, C):
    return (s-(X+C))*(1/(s*sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(np.log(s)-mi_ST)**2/(2*sigma_ST**2))

def ret_ct(s, C):
    return -C*(1/(s*sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(np.log(s)-mi_ST)**2/(2*sigma_ST**2))

def diferenca_esperancas(C):
    Pt = quad(ret_ct, 0, X, args=(C))[0]
    Pp = quad(ret_lin, X, X + C, args=(C))[0]
    L = quad(ret_lin, X + C, np.inf, args=(C))[0]
    D = abs(L + Pt + Pp)
    return D

C_freitas = minimize_scalar(diferenca_esperancas).x * np.exp(-r*mi_ST*T)

In [25]:
valores = pd.DataFrame({'Bachelier': [C_bachelier], 'Black-Scholes': [C_BS], 'Freitas': [C_freitas]})
valores

,Bachelier,Black-Scholes,Freitas
0,0.070649,4.104994,4.613341


<center> <h1> <b>Gráfico Interativo</b> </h1> </center>

In [26]:
def serie_bachelier(X, S, T, sigma):
    serie = []
    for t in range(1, T+11):
        D = (S-X)/(sigma*np.sqrt(t))
        serie.append((S-X)*norm.cdf(D) + sigma*np.sqrt(t)*norm.pdf(D))
    return serie

def serie_BS(X, S, T, sigma):
    serie = []
    for t in range(1, T+11):
        d1 = (np.log(S/X) + (r + (sigma**2)/2)*t) / (sigma*np.sqrt(t))
        d2 = d1 - sigma*np.sqrt(t)
        serie.append(S*norm.cdf(d1) - X*np.exp(-r*t)*norm.cdf(d2))
    return serie

def serie_freitas(X, S, T, sigma, mi):
    serie = []
    for t in range(1, T+11):
        mi_ST = np.log(S) + t*mi
        sigma_ST = np.sqrt(t)*sigma

        def ret_lin(s, C):
            return (s-(X+C))*(1/(s*sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(np.log(s)-mi_ST)**2/(2*sigma_ST**2))

        def ret_ct(s, C):
            return -C*(1/(s*sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(np.log(s)-mi_ST)**2/(2*sigma_ST**2))

        def diferenca_esperancas_interna(C):
            Pt = quad(ret_ct, 0, X, args=(C))[0]
            Pp = quad(ret_lin, X, X + C, args=(C))[0]
            L = quad(ret_lin, X + C, np.inf, args=(C))[0]
            D = abs(L + Pt + Pp)
            return D

        serie.append(minimize_scalar(diferenca_esperancas_interna).x)
    return serie

In [27]:
# Gráfico Interativo
def update_plot(X):
    x = np.linspace(1, T+10, T+10)
    y_bachelier = serie_bachelier(X, S, T, sigma)
    y_BS = serie_BS(X, S, T, sigma)
    y_freitas = serie_freitas(X, S, T, sigma, mi)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y_bachelier, mode='lines', name=f'Bachelier'))
    fig.add_trace(go.Scatter(x=x, y=y_BS, mode='lines', name=f'Black-Scholes'))
    fig.add_trace(go.Scatter(x=x, y=y_freitas, mode='lines', name=f'Freitas'))

    fig.update_layout(
        title=f'Gráfico de Precificação para X = {X/S:.2f}S',
        xaxis_title='Períodos',
        yaxis_title='Preço da Opção',
        yaxis_range=[0, 0.1*S],
        template='plotly_dark')
    fig.show()
    
interact(update_plot, X=(0.8*S, 1.2*S, 0.1))

interactive(children=(FloatSlider(value=58.14400024414063, description='X', max=69.81600036621093, min=46.5440…

<function __main__.update_plot(X)>

In [28]:
def ret_lin(s, P):
    return (s-(X+P))*(1/(sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(s-mi_ST)**2/(2*sigma_ST**2))

def ret_ct(s, P):
    return -P*(1/(sigma_ST*np.sqrt(2*np.pi)))*np.exp(-(s-mi_ST)**2/(2*sigma_ST**2))

def diferenca_esperancas(P):
    Pt = quad(ret_ct, X, np.inf, args=(P))[0]
    Pp = quad(ret_lin, X - P, X, args=(P))[0]
    L = quad(ret_lin, 0, X - P, args=(P))[0]
    D = abs(L + Pt + Pp)
    return D

P_freitas = minimize_scalar(diferenca_esperancas).x * np.exp(-r*mi_ST*T)
P_freitas

-54.11645896923178